In [125]:
import pandas as pd
import altair as alt
alt.data_transformers.disable_max_rows()



DataTransformerRegistry.enable('default')

In [126]:
data = pd.read_csv("snirh_clean.csv")
data= data[(data.tipo_de_infraestrutura == "bacia") & (data.medida == "percentagem")]


In [127]:
data["resumo_infraestrutura"] = pd.to_numeric(data["resumo_infraestrutura"], errors='coerce')
idx = data.groupby(["medida","tipo_de_infraestrutura","nome_infraestrutura"]).resumo_infraestrutura.idxmin()
min_ = data.loc[idx].sort_values(by=['data'])
min_["min"] = "minimum"
min_.rename( columns={"Unnamed: 0" :'index'}, inplace=True )
data.rename( columns={"Unnamed: 0" :'index'}, inplace=True )

data_minimos = data.join(min_, on="index", rsuffix="_r").sort_values(by=['min']).reset_index()

In [128]:
bacias = ['ARADE', 'AVE', 'CÁVADO/RIBEIRAS COSTEIRAS', 'DOURO',
           'GUADIANA', 'LIMA', 'MIRA', 'MONDEGO', 'RIBEIRAS DO ALGARVE',
           'RIBEIRAS DO OESTE', 'SADO', 'TEJO']

In [129]:
chart = alt.Chart(data_minimos).mark_area(strokeWidth=1,    
                          color="#5bc0be",
                          interpolate='step',
                          opacity=0.5
                         ).encode(
    x=alt.X('data:T', axis=alt.Axis(tickCount={"interval": "year", "step": 1},title=" ")),
    y=alt.Y('resumo_infraestrutura:Q', axis=None),
    text=alt.Text('min(resumo_infraestrutura):Q'),
    tooltip="resumo_infraestrutura"
).properties(
    width=2000
)

line = alt.Chart(data_minimos).mark_rule(color="red").encode(
    alt.X("data:T")
).transform_filter( (datum.min == "minimum")
)


(chart + line).facet(
    row="nome_infraestrutura",
    spacing=0
)

alt.FacetChart(...)